# Importar librerias y perfil AWS

In [4]:
from jinja2 import Template
import pytz
from datetime import datetime, timedelta
import awswrangler as wr
import pandas as pd
import os
import re
import shutil

from extra.config import awsconfig
from extra.utils import (extract, load, utils)

awsconfig.aws_config_profile('NEQUI-DataBusinessAnalyst-Role-044879804046')

Perfil AWS configurado: NEQUI-DataBusinessAnalyst-Role-044879804046


In [5]:
dict_data_sql_templates = {

    # Parametos generales Athena
    'workgroup': 'delfos',

    # Parametros consulta SQL 1
    'finacle_producto':
    {
        'database':'co_delfos_clientes_raw_pdn_rl',
        'table': 'producto_hist'
    },
}
# Fecha de consulta Hora Colombia
colombia_tz = pytz.timezone("America/Bogota") 
time_co = datetime.now(colombia_tz) - timedelta(days=1)

dict_params_partition = {
    # Año, mes, dia
    'year_co': str(time_co.year),
    'month_co': str(time_co.month).zfill(2),
    'day_co': str(time_co.day).zfill(2)
}

# Consulta Finacle-Producto de cada usuario

In [6]:
# Cargar template query SQL
sql_template_finacle_producto = Template(open('../utils/templates_querys/finacle_productos.sql', 'r').read())

# Parametros de entrada Query SQL
query_finacle_producto_params = {   'database': dict_data_sql_templates['finacle_producto']['database'],
                                    'table': dict_data_sql_templates['finacle_producto']['table'], 
                                    'year_co': f"'{dict_params_partition['year_co']}'",
                                    'month_co': f"'{dict_params_partition['month_co']}'",
                                    'day_co':  f"'{dict_params_partition['day_co']}'"
                                }
# Render template Jinja SQL
sql_finacle_producto = sql_template_finacle_producto.render(query_finacle_producto_params)

# Df con los datos consultados desde Athena
df_finacle_producto = wr.athena.read_sql_query(sql=sql_finacle_producto, 
                                              database=dict_data_sql_templates['finacle_producto']['database'], 
                                              workgroup=dict_data_sql_templates['workgroup'],
                                               ctas_approach=False)
# Mensaje de resultado
print(f"Consulta: \n\n {sql_finacle_producto}\n\n Completada exitosamente!!!")

Consulta: 

 SELECT
    trim(both ' ' FROM replace(replace(numero_celular, '+57(', ''), ')', '')) as numero_celular
FROM co_delfos_clientes_raw_pdn_rl.producto_hist
WHERE true
        AND year='2025'
        AND month = '01'
        AND day = '26'
        AND CAST(valor_saldo_cuenta as DECIMAL(18, 0)) > 30000000
        ORDER BY CAST(valor_saldo_cuenta as DECIMAL(18, 0)) DESC
limit 1000

 Completada exitosamente!!!


In [7]:
df_finacle_producto

,numero_celular


In [8]:
# Crear el string con el formato solicitado
data_string = '\n'.join([f'cel.vip,cel={row.numero_celular} 1' for index, row in df_finacle_producto.iterrows()])

print(data_string)

Mandar la data a Dynatrace

In [9]:
import os
from dotenv import load_dotenv
import requests

# Lectura desde .env
load_dotenv()

# Obtener el token
api_token = os.getenv('API_TOKEN')

endpoint_url = "https://iyl01250.live.dynatrace.com/api/v2/metrics/ingest"
data = 'cel.vip,cel=3006402129, id=a23 1 \ncel.vip,cel=123 1'
headers = {
    "Authorization": f"Api-Token {api_token}"
}

response = requests.post(endpoint_url, data=data_string, headers=headers)
print(response.status_code)
print(response.json())


400
{'linesOk': 0, 'linesInvalid': 0, 'error': {'code': 400, 'message': 'empty request', 'invalidLines': []}, 'warnings': None}
